In [1]:
import sys
import gc
sys.path.append('../../')

from scoring.event_detection_matrix import competition_score
from models.mlp.mlp import PyTorchMLP, LightningModel

from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

## Load Model

In [2]:
checkpoint_path = "../../models/neural-nappers/wv1kmbtj/checkpoints/epoch=4-step=871060.ckpt"
model = LightningModel.load_from_checkpoint(checkpoint_path)

## Load Data

In [3]:
df_train_series = pd.read_parquet('../../data/processed/handcrafted-features/train_series_split.parquet')
df_validation_series = pd.read_parquet('../../data/processed/handcrafted-features/validation_series_split.parquet')
df_validation_events = pd.read_csv('../../data/processed/handcrafted-features/validation_events_split.csv')

In [4]:
df_validation_events = df_validation_events[df_validation_events.step.notnull()]
df_validation_events

,series_id,night,event,step,timestamp,num_series_id
0,062dbd4c95e6,1,onset,7872.0,2018-08-22T23:11:00-0400,7
1,062dbd4c95e6,1,wakeup,14484.0,2018-08-23T08:22:00-0400,7
6,062dbd4c95e6,4,onset,60720.0,2018-08-26T00:35:00-0400,7
7,062dbd4c95e6,4,wakeup,68400.0,2018-08-26T11:15:00-0400,7
8,062dbd4c95e6,5,onset,77304.0,2018-08-26T23:37:00-0400,7
...,...,...,...,...,...,...
2955,fcca183903b7,33,wakeup,565824.0,2019-04-28T06:52:00-0400,276
2956,fcca183903b7,34,onset,577344.0,2019-04-28T22:52:00-0400,276
2957,fcca183903b7,34,wakeup,584052.0,2019-04-29T08:11:00-0400,276
2958,fcca183903b7,35,onset,595344.0,2019-04-29T23:52:00-0400,276


In [5]:
#df_validation_series = df_validation_series[df_validation_series.series_id == '038441c925bb']
#df_validation_events = df_validation_events[df_validation_events.series_id == '038441c925bb']
print(df_validation_series.shape)

(16484071, 21)


## Prepare Data

In [6]:
FEATURES = ['anglez', 'enmo',
            'hour',
            'anglez_abs', 'anglez_diff', 'enmo_diff', 'anglez_x_enmo',
            'anglez_rolling_mean', 'enmo_rolling_mean', 'anglez_rolling_max', 'enmo_rolling_max', 'anglez_rolling_min',
            'anglez_rolling_std', 'enmo_rolling_std']

LABEL = ['awake']

In [7]:
X_train = df_train_series[FEATURES].astype('float32')
scaler = StandardScaler()
scaler.fit(X_train)

del X_train
gc.collect()

198

In [8]:
X_validation = df_validation_series[FEATURES].astype('float32')
y_validation = df_validation_series[LABEL].astype('int64')

X_validation = scaler.transform(X_validation)

X_validation = torch.from_numpy(X_validation)
y_validation = torch.from_numpy(y_validation.to_numpy()).squeeze(1)

validation_dataset = TensorDataset(X_validation, y_validation)
validation_dataloader = DataLoader(validation_dataset, batch_size=10000)

## Predict

In [9]:
def predict(batch):
    X, y = batch
    with torch.no_grad():
        logits = model(X)
    label = torch.argmax(logits, dim=-1)
    confidence = torch.softmax(logits, dim=-1)
    confidence_0 = confidence[:, 0]
    confidence_1 = confidence[:, 1]
    return label, confidence_0, confidence_1

In [10]:
model.eval()
label_list = []
confidence_0_list = []
confidence_1_list = []

for index, batch in enumerate(tqdm(validation_dataloader)) :
    label, confidence_0, confidence_1 = predict(batch)
    
    label_list.append(label)
    confidence_0_list.append(confidence_0)
    confidence_1_list.append(confidence_1)      

100%|██████████████████████████████████████████████████████████████████████████████| 1649/1649 [04:22<00:00,  6.28it/s]


In [11]:
label_list = torch.cat(label_list).numpy()
confidence_0_list = torch.cat(confidence_0_list).numpy()
confidence_1_list = torch.cat(confidence_1_list).numpy()

In [12]:
df_validation_series['prediction_class'] = label_list
df_validation_series['prediction_confidence_0'] = confidence_0_list
df_validation_series['prediction_confidence_1'] = confidence_1_list

In [13]:
df_validation_series = df_validation_series[['series_id', 'step', 'timestamp', 'awake', 'prediction_class', 'prediction_confidence_0', 'prediction_confidence_1']]
df_validation_series

,series_id,step,timestamp,awake,prediction_class,prediction_confidence_0,prediction_confidence_1
1635640,062dbd4c95e6,0,2018-08-22T12:15:00-0400,1,1,0.000166,0.999834
1635641,062dbd4c95e6,1,2018-08-22T12:15:05-0400,1,1,0.000134,0.999866
1635642,062dbd4c95e6,2,2018-08-22T12:15:10-0400,1,1,0.000144,0.999856
1635643,062dbd4c95e6,3,2018-08-22T12:15:15-0400,1,1,0.000116,0.999884
1635644,062dbd4c95e6,4,2018-08-22T12:15:20-0400,1,1,0.000141,0.999859
...,...,...,...,...,...,...,...
88954459,fcca183903b7,620635,2019-05-01T10:59:35-0400,1,1,0.124150,0.875850
88954460,fcca183903b7,620636,2019-05-01T10:59:40-0400,1,1,0.124150,0.875850
88954461,fcca183903b7,620637,2019-05-01T10:59:45-0400,1,1,0.124150,0.875850
88954462,fcca183903b7,620638,2019-05-01T10:59:50-0400,1,1,0.124150,0.875850


## Extract Events

In [14]:
def get_events_smoothed(test_series) :
    series_ids = test_series['series_id'].unique()
    events = []

    for idx in tqdm(series_ids):
        # Collecting sample and normalizing features
        X = test_series[test_series.series_id == idx]
                
        smoothing_length = 12 * 60 # 30 Minutes
        # We average the confidence, that the participant is awake
        X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
        X["asleep"] = X["prediction_confidence_0"].rolling(smoothing_length, center=True).mean().bfill().ffill()

        # Binarize the asleep column
        X["asleep"] = X["asleep"].round()

        # Getting predicted onset and wakeup time steps
        pred_onsets = X[X['asleep'].diff() > 0]['step'].tolist() # diff is > 0 if it changes from 0 (awake) to 1 (asleep)
        pred_wakeups = X[X['asleep'].diff() < 0]['step'].tolist() # diff is < 0 if it changes from 1 (asleep) to 0 (awake)
     
        if len(pred_onsets) > 0:

            # Ensuring all predicted sleep periods begin and end
            if min(pred_wakeups) < min(pred_onsets):
                pred_wakeups = pred_wakeups[1:]

            if max(pred_onsets) > max(pred_wakeups):
                pred_onsets = pred_onsets[:-1]

            # Keeping sleep periods longer than 30 minutes
            sleep_periods = [(onset, wakeup) for onset, wakeup in zip(pred_onsets, pred_wakeups) if wakeup - onset >= 12 * 30]

            for onset, wakeup in sleep_periods:
                # We take the score
                score = 1 - X[(X['step'] >= onset) & (X['step'] < wakeup)]['confidence_awake'].mean()

                # Adding sleep event to dataframe
                onset_row = {'row_id': len(events), 'series_id': idx, 'step': onset, 'event': 'onset', 'score': score}                
                events.append(onset_row)

                wakeup_row = {'row_id': len(events), 'series_id': idx, 'step': wakeup, 'event': 'wakeup', 'score': score}
                events.append(wakeup_row)

    return pd.DataFrame(events)

In [15]:
predicted_validation_events = get_events_smoothed(df_validation_series)

  0%|                                                                                           | 0/54 [00:00<?, ?it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["asleep"] = X["prediction_confidence_0"].rolling

 11%|█████████▏                                                                         | 6/54 [00:04<00:37,  1.26it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["asleep"] = X["prediction_confidence_0"].rolling

 22%|██████████████████▏                                                               | 12/54 [00:09<00:32,  1.27it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["asleep"] = X["prediction_confidence_0"].rolling

 33%|███████████████████████████▎                                                      | 18/54 [00:14<00:27,  1.33it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["asleep"] = X["prediction_confidence_0"].rolling

 44%|████████████████████████████████████▍                                             | 24/54 [00:18<00:22,  1.33it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["asleep"] = X["prediction_confidence_0"].rolling

 56%|█████████████████████████████████████████████▌                                    | 30/54 [00:23<00:18,  1.31it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["asleep"] = X["prediction_confidence_0"].rolling

 67%|██████████████████████████████████████████████████████▋                           | 36/54 [00:28<00:14,  1.24it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["asleep"] = X["prediction_confidence_0"].rolling

 78%|███████████████████████████████████████████████████████████████▊                  | 42/54 [00:32<00:09,  1.29it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["asleep"] = X["prediction_confidence_0"].rolling

 89%|████████████████████████████████████████████████████████████████████████▉         | 48/54 [00:37<00:04,  1.32it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
C:\Users\timon\AppData\Local\Temp\ipykernel_28032\2515621411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["asleep"] = X["prediction_confidence_0"].rolling

100%|██████████████████████████████████████████████████████████████████████████████████| 54/54 [00:41<00:00,  1.29it/s]


In [16]:
predicted_validation_events

,row_id,series_id,step,event,score
0,0,062dbd4c95e6,7839,onset,0.849435
1,1,062dbd4c95e6,14598,wakeup,0.849435
2,2,062dbd4c95e6,60330,onset,0.833066
3,3,062dbd4c95e6,66662,wakeup,0.833066
4,4,062dbd4c95e6,77258,onset,0.853671
...,...,...,...,...,...
2037,2037,fcca183903b7,565867,wakeup,0.903280
2038,2038,fcca183903b7,577381,onset,0.911127
2039,2039,fcca183903b7,583940,wakeup,0.911127
2040,2040,fcca183903b7,595288,onset,0.866994


## Evaluate

### Accuracy

In [17]:
accuracy_score(df_validation_series['awake'].values, df_validation_series['prediction_class'].values)

0.9237835119734682

### Competition Score

In [18]:
competition_score(df_validation_events, predicted_validation_events)

0.3705473239674369

In [19]:
df_validation_series.to_parquet('./data/predicted_series_mlp_new_full.parquet')